In [7]:
import os
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient

# Set up download directory
DATA_DIR_PATH = os.path.join(os.getcwd(), "data")
if not os.path.exists(DATA_DIR_PATH):
    os.makedirs(DATA_DIR_PATH)

In [8]:
def get_blob_service_client_account_key():
    account_url = "https://saqdiveassignments.blob.core.windows.net/"
    # credential = DefaultAzureCredential()

    # Create the BlobServiceClient object
    blob_service_client = BlobServiceClient(account_url, credential=None)

    return blob_service_client


# def download_blobs_to_dir(blob_service_client: BlobServiceClient, container_name, file_name):
#     container_client = blob_service_client.get_container_client(container=container_name)
#     for blob in container_client.list_blobs():
#         with open(file=os.path.join(DATA_DIR_PATH, file_name), mode="wb") as sample_blob:
#             download_stream = blob.download_blob()
#             sample_blob.write(download_stream.readall())

In [9]:
service = get_blob_service_client_account_key()
container_client = service.get_container_client(container="dataengineerfiles")
for blob in container_client.list_blob_names():
    print(blob)

cast.tsv
crew.tsv
names.tsv
ratings.tsv
titles.tsv
